## Christina Chappell
### ArcPy Lab 1 
### February 1, 2017

#### Total Hours: A little under 20 hours over the past week.

### part 1. Creating geoJSON files for each county in WA.


In [1]:
## This lets me use arcpy within this python setting.
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.4\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.4\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.4\\ArcToolbox\\Scripts')
import arcpy

#### This code will let me convert the shapefiles into geoJSON files later on

In [2]:
## This lets me make geoJSON files
from subprocess import call
import os

os.environ["GDAL_DATA"] = "C:\\OSGeo4W64\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W64\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W64\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W64\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W64\\apps\\msys\\bin;C:\\OSGeo4W64\\apps\\Python27\\Scripts"


### This creates the separate shapefiles for each county from the original provided shapefile
#### To create the geoJSON files for each county based on just the block data I changed methods a few times. I knew I had to iterate through both files to make sure that the county names are with the correct blocks, at first I created a new field in the saep_bg10 file but that kept taking too long to implement. I used a nested for loop to get all the blocks with a search cursor and an update cursor. When I changed course I made both search cursors and created the new shape files all at once by using an insert cursor. The part of the assignment was done in small steps that worked into the last step to make the code more complicated but being able to do it all at once.

In [ ]:
#Setting up the workspace, and creating copy files to make the altering of fields easier
arcpy.env.workspace = "U:\\Geography\\GEOG458" 
arcpy.env.overwriteOutput = True

arcpy.management.CopyFeatures(
    "U:\\Geography\\Geog458\\saep_bg11\\saep_bg10.shp",
    "U:\\Geography\\Geog458\\saep_bg11\\saep_bg10_copy.shp")

#This calls the two files provided from the assignment
fc = "U:\\Geography\\GEOG458\\saep_bg11\\saep_bg10_copy.shp"
fips = "U:\\Geography\\GEOG458\\arcpylab1\\WashingtonFIPS.dbf"

#which fields to use and copy
fields_to_use =["SHAPE@", "COUNTYFP10", "POP2016"]
#cursor to search over the counties and connect the correct
#  rows to the county files
cursor_wa = arcpy.da.SearchCursor(fips, ["FIPSCounty", "CountyName"])

for row in cursor_wa:
    filename = row[1] + ".shp"
    new_table = arcpy.CreateFeatureclass_management(
        "U:\\Geography\\GEOG458\\geojson\\", filename, "polygon", fc)
    #search the saep file for the correct rows
    cursor_saep = arcpy.da.SearchCursor(fc, fields_to_use)
    #cursor to insert the rows to the new shape files
    cursor_to_insert = arcpy.da.InsertCursor(new_table, fields_to_use)
    
    #Inserts the block row into the correct shapfile
    for row2 in cursor_saep:  
        if (row[0] == row2[1]):      
            cursor_to_insert.insertRow(row2)

del cursor_wa
del cursor_saep
del cursor_to_insert


### Creating the geoJSON files
#### This part I struggled with because the I had the incorrect code copied from canvas, once I got the correct code it was lot easier to use the search cursor and create the geoJSON files correctly.

In [11]:
arcpy.env.workspace = "U:/Geography/GEOG458" 
arcpy.env.overwriteOutput = True
#To go through all the counties and make the correct files
cursor = arcpy.da.SearchCursor(fips, "CountyName")
for row in cursor:
    geojson_file = 'U:\\Geography\\GEOG458\\geojson\\file\\' + row[0] + '.geojson'
    shape_file = 'U:\\Geography\\GEOG458\\geojson\\' + row[0] + '.shp'
    
    call(['C:\\OSGeo4W64\\bin\\ogr2ogr',
          '-f','GeoJSON','-t_srs','WGS84',
         '-s_srs','EPSG:2927', 
        geojson_file, 
          shape_file])
del cursor

### Part 2: Top 10 populations by County
#### This part took less amount of time. My thinking on it was to have another nested for loop with cursors to go over each block and get the population from that block. I add the populations together from the same counties and append a list I created with the total county population. Then I sort the list and print out the top 10 populations.

In [30]:
#Create an empty list to put all of the populations for the counties
my_list = []
pop = 0

#Use 2 cursors and a nested for loop again to add each blocks pop to the county pop
fipcursor = arcpy.da.SearchCursor(fips, ["FIPSCounty", "CountyName"])
for row in fipcursor:
    popcursor = arcpy.da.SearchCursor(fc, ['COUNTYFP10', 'POP2016'])
    for row2 in popcursor:
        if row[0] == row2[0]:
            pop = pop + row2[1]
            pop = int(pop)
    my_list.append(pop)
    pop = 0

#this will print out the list of top 10 populations. Descending from 10 to 1
my_list.sort(reverse = True)
n = 9
while n >= 0:
    print  str(n + 1) + ": " + str(my_list[n])
    n = n - 1
  

10: 190429
9: 212484
8: 250819
7: 262513
6: 272611
5: 460870
4: 492378
3: 772609
2: 844211
1: 2104403
